In [1]:
import pandas as pd
import numpy as np
import PIL
import folium
import utilHyp

In [2]:
def haversineFunction(cord1, cord2):

    p1 = np.sin((cord2[0]-cord1[0])/2)**2
    p2 = np.cos(cord1[0])
    p3 = np.cos(cord2[0])
    p4 = np.sin((cord2[1]-cord1[1])/2)**2
    p5 = p2 * p3 * p4
    h = p1 + p5

    r = 6371
    return(2*r*np.arcsin(np.sqrt(h)))


### Recommendation

We will recommend the spots of interest to a user based on location where User's walk was pictured and the following information:

1.Transportation Mode 
- Walk
- Transit
- Driving

2.User's Interests
- Enterntainment
- Night Out & Drink
- Food
- Tourism & Sightseeing
- Historic Sight

Taking a final look at processed data is needed to determine what and where we recommend.

In [3]:
tourism = pd.read_json("./Prediction Data/tour_final")
historic = pd.read_json("./Prediction Data/hist_final")
food = pd.read_json("./Prediction Data/food_final")
night = pd.read_json("./Prediction Data/night_final")
subway = pd.read_json("./Prediction Data/subway_final")
enterntainment = pd.read_json("./Prediction Data/entr_final")

In [4]:
tourism.type.unique()

array(['information', 'attraction', 'hostel', 'artwork', 'guest_house',
       'museum', 'camp_site', 'hotel', 'viewpoint', 'apartment',
       'picnic_site', 'gallery', 'theme_park', 'motel', 'caravan_site',
       'winery', 'lookout', 'Plane_Spotting_Platform', 'house'],
      dtype=object)

In [5]:
historic.type.unique()

array(['monument', 'memorial', 'ruins', 'yes', 'marker', 'aircraft',
       'boundary_stone', 'milepost', 'milestone', 'building', 'tree',
       'church', 'ship', 'locomotive'], dtype=object)

In [6]:
food.amenity.unique()

array(['restaurant', 'cafe', 'fast_food', 'ice_cream', 'bistro',
       'food_court'], dtype=object)

In [7]:
night.amenity.unique()

array(['nightclub', 'pub', 'bar', 'events_venue'], dtype=object)

In [8]:
enterntainment.amenity.unique()

array(['theatre', 'cinema', 'arts_centre', 'townhall', 'gambling',
       'leisure', 'internet_cafe', 'lounge', 'casino'], dtype=object)

## Recommendation System Initial Setup
- Take mode of tour & transportation
- Reccommend spots close depending on interest & distance!
- Embed images (if exist) into spot 'pop ups'

In [10]:
def setup(tour_mode, hints, move_mode, imageWalk):
    if tour_mode.lower() == 'tourism':
        plotmap('tourism', hints, move_mode, imageWalk)
    elif tour_mode.lower() == 'historic':
        plotmap('historic', hints, move_mode, imageWalk)
    elif tour_mode.lower() == 'food':
        plotmap('food', hints, move_mode, imageWalk)
    elif tour_mode.lower() == 'night':
        plotmap('night', hints, move_mode, imageWalk)
    elif tour_mode.lower() == 'enterntainment':
        plotmap('enterntainment', hints, enterntainment, imageWalk)

In [69]:
def plotmap(data, hints, move_mode, user_feed):
    
    map_full = folium.Map(
        location=[49.28, -123.12],
        zoom_start = 12
    )
    
    if move_mode == 'walk':
        threshold = 500/10
    elif move_mode == 'transit':
        threshold = 1500/10
    elif move_mode == 'driving':
        threshold = 3000/10
    
    
    # Find recommendation for each image on a path.
    for each in user_feed:

        # Mark User's Location
        # WHERE IS THE USER CLOSE TO?
        folium.Marker([user_feed[0], user_feed[1]], popup = "YOU", icon = folium.Icon(color = 'green')).add_to(map_full)

        # Show a User closest subway stations if traveling by transit.
        if(move_mode == 'transit'):
            subway_results = subway
            subway_results['distance'] = haversineFunction([subway_results['lat'], subway_results['lon']], user_feed)
            subway_results = subway_results[subway_results['distance'] < threshold]
            subway_results = subway_results.iterrows()
            for idx, spot in subway_results:
                folium.Marker([spot['lat'], spot['lon']], popup = spot['name'], icon = folium.Icon(color = 'blue')).add_to(map_full)

        
        if data == 'tourism':
                        
            # Recommend everything relevant from the category if no hints given.
            if(len(hints) == 0):
                results = tourism
                results['distance'] = haversineFunction([results['lat'], results['lon']], user_feed)
                results = results[results['distance'] < threshold]
                results = results.iterrows()
                for idx, spot in results:
                    folium.Marker([spot['lat'], spot['lon']], popup = spot['name'], icon = folium.Icon(color = 'red')).add_to(map_full)

            # Priority Recommendations. Hints.
            elif(len(hints) > 0):
                for hint in hints:
                    results = tourism[tourism['type'] == hint]
                    results['distance'] = haversineFunction([results['lat'], results['lon']], user_feed)
                    results = results[results['distance'] < threshold]
                    results = results.iterrows()
                    for idx, spot in results:
                        folium.Marker([spot['lat'], spot['lon']], popup = spot['name'], icon = folium.Icon(color = 'red')).add_to(map_full)


                        
        elif data == 'historic':
            
            # Recommend everything relevant from the category if no hints given.
            if(len(hints) == 0):
                results = historic
                results['distance'] = haversineFunction([results['lat'], results['lon']], user_feed)
                results = results[results['distance'] < threshold]
                results = results.iterrows()
                for idx, spot in results:
                    folium.Marker([spot['lat'], spot['lon']], popup = spot['name'], icon = folium.Icon(color = 'purple')).add_to(map_full)


            # Priority Recommendations. Hints.
            elif(len(hints) > 0):
                for hint in hints:
                    results = historic
                    results = results[results['type'] == hint]
                    results['distance'] = haversineFunction([results['lat'], results['lon']], user_feed)
                    results = results[results['distance'] < threshold]
                    results = results.iterrows()
                    for idx, spot in results:
                        folium.Marker([spot['lat'], spot['lon']], popup = spot['name'], icon = folium.Icon(color = 'purple')).add_to(map_full)

            
            
        elif data == 'food':
            # Recommend everything relevant from the category if no hints are given.
            if(len(hints) == 0):
                results = food
                results['distance'] = haversineFunction([results['lat'], results['lon']], user_feed)
                results = results[results['distance'] < threshold]
                results = results.iterrows()
                for idx, spot in results:
                    if spot['cuisine'] != None:
                        folium.Marker([spot['lat'], spot['lon']], popup = spot['name'] + "\n\n(" + spot['amenity'] + " - " + spot['cuisine'] + ")", icon = folium.Icon(color = 'red')).add_to(map_full)
                    else:
                        folium.Marker([spot['lat'], spot['lon']], popup = spot['name'] + "\n\n(" + spot['amenity'] + ")", icon = folium.Icon(color = 'red')).add_to(map_full)


            # Priority Recommendations. Hints.
            elif(len(hints) > 0):
                for hint in hints:
                    results = food
                    results = results[results['amenity'] == hint]
                    results['distance'] = haversineFunction([results['lat'], results['lon']], user_feed)
                    results = results[results['distance'] < threshold]
                    results = results.iterrows()
                    for idx, spot in results:
                        if spot['cuisine'] != None:
                            folium.Marker([spot['lat'], spot['lon']], popup = spot['name'] + "\n\n(" + spot['amenity'] + " - " + spot['cuisine'] + ")", icon = folium.Icon(color = 'red')).add_to(map_full)
                        else:
                            folium.Marker([spot['lat'], spot['lon']], popup = spot['name'] + "\n\n(" + spot['amenity'] + ")", icon = folium.Icon(color = 'red')).add_to(map_full)
                            
                         

        elif data == 'night':
            # Recommend everything relevant from the category if no hints given.
            if(len(hints) == 0):
                results = night
                results['distance'] = haversineFunction([results['lat'], results['lon']], user_feed)
                results = results[results['distance'] < threshold]
                results = results.iterrows()
                for idx, spot in results:
                    if spot['amenity'] != None:
                        folium.Marker([spot['lat'], spot['lon']], popup = spot['name'] + "\n\n(" + spot['amenity'] + ")", icon = folium.Icon(color = 'cadetblue')).add_to(map_full)
                    else:
                        folium.Marker([spot['lat'], spot['lon']], popup = spot['name'], icon = folium.Icon(color = 'cadetblue')).add_to(map_full)
                        
                        
            # Priority Recommendations. Hints.
            elif(len(hints) > 0):
                for hint in hints:
                    results = night
                    results = results[results['amenity'] == hint]
                    results['distance'] = haversineFunction([results['lat'], results['lon']], user_feed)
                    results = results[results['distance'] < threshold]
                    results = results.iterrows()
                    for idx, spot in results:
                        if spot['amenity'] != None:
                            folium.Marker([spot['lat'], spot['lon']], popup = spot['name'] + "\n(" + spot['amenity'] + ')', icon = folium.Icon(color = 'cadetblue')).add_to(map_full)
                        else:
                            folium.Marker([spot['lat'], spot['lon']], popup = spot['name'], icon = folium.Icon(color = 'cadetblue')).add_to(map_full)
                            
                        

        elif data == 'enterntainment':
            # Recommend everything relevant from the category if no hints given.
            if(len(hints) == 0):
                results = enterntainment
                results['distance'] = haversineFunction([results['lat'], results['lon']], user_feed)
                results = results[results['distance'] < threshold]
                results = results.iterrows()
                for idx, spot in results:
                    if result['amenity'] != None:
                        folium.Marker([spot['lat'], spot['lon']], popup = spot['name'] + '\n\n(' + spot['amenity'] + ")", icon = folium.Icon(color = 'red')).add_to(map_full)

            # Priority Recommendations. Hints.
            elif(len(hints) > 0):
                for hint in hints:
                    results = enterntainment
                    results = results[results['amenity'] == hint]
                    results['distance'] = haversineFunction([results['lat'], results['lon']], user_feed)
                    results = results[results['distance'] < threshold]
                    results = results.iterrows()
                    for idx, spot in results:
                        if result['amenity'] != None:
                            folium.Marker([spot['lat'], spot['lon']], popup = spot['name'] + '\n\n(' + spot['amenity'] + ")", icon = folium.Icon(color = 'red')).add_to(map_full)


    display(map_full);

In [68]:
enterntainment

,WikiData,amenity,image,lat,lon,name,tags
0,Q38377943,theatre,None,49.245578,-123.185447,Dunbar Theatre,"{'addr:housenumber': '4555', 'addr:street': 'D..."
1,Q38378154,theatre,None,49.203711,-123.137523,Metro Theatre,"{'addr:housenumber': '1370', 'website': 'https..."
2,Q38378234,theatre,None,49.281101,-123.098370,The Rickshaw,"{'addr:housenumber': '254', 'website': 'http:/..."
3,Q38378292,cinema,None,49.281998,-123.124346,Scotiabank Theatre Vancouver,"{'addr:housenumber': '900', 'website': 'https:..."
4,Q38378369,theatre,None,49.279991,-123.116458,The Centre For Performing Arts,"{'addr:province': 'BC', 'addr:housenumber': '7..."
...,...,...,...,...,...,...,...
73,None,townhall,None,49.038055,-123.097832,Tsawwassen First Nation Administration Offices,"{'addr:housenumber': '1926', 'website': 'http:..."
74,None,lounge,None,49.197137,-123.175708,Maple Leaf Lounge,{'operator': 'Air Canada'}
75,None,lounge,None,49.196528,-123.177766,Sakura Lounge,{'operator': 'JAL'}
76,None,lounge,None,49.195936,-123.178125,Plaza Premium Lounge,{}


## Testing

Please Input your CHOICE for the Tour. You can pass HINTS to prioritize the spots as a list.
- Tourism 
        information, attraction, hostel, artwork, guest_house,
        museum, camp_site, hotel, viewpoint, apartment,
        picnic_site, gallery, theme_park, motel, 'caravan_site,
        winery, lookout, Plane_Spotting_Platform, 'house
- Historic
        monument, memorial, ruins, aircraft, boundary_stone, ship,
        milepost, milestone, building, tree, church, locomotive
- Food
        restaurant, cafe, fast_food, ice_cream, bistro,
        food_court
        
- Night Enterntainment
        nightclub, pub, bar, events_venue

- Enterntainment
        theatre, cinema, arts_centre, townhall, gambling,
        leisure, internet_cafe, lounge, casino

AND your preffered transportation mode (Some recommendations will vary):
- Walk
- Drive
- Subway


In [70]:
setup('food', [], 'walk', [49.2781, -123.0840])